In [1]:
from aectemplater_ui.apps.constants import (
    EUI_DATA,
    OBJECT_ID_BUILDING_AREA,
    OBJECT_ID_OPERATIONAL_ENERGY_TARGET,
    OBJECT_ID_PREDICTED,
    OBJECT_ID_RECORDED,
    TEST_PROJECT_REVISION_ID,
)
from netzero_metrics.constants import nzdata
from aectemplater_ui.apps.netzero import (
    get_project_building_data,
    get_area_data,
    get_project_energy_targets,
    get_area_and_target_summaries,
)
from netzero_metrics.calcs import get_project_targets

import pandas as pd
from netzero_metrics.calcs import (
    get_khwr_per_m2_gia_only,
    attach_target_year_to_area,
    get_area_summary,
    get_area_weights,
    get_project_area_weights,
    get_all_buildings_yearly_eui_uknzcb_targets,
    get_project_targets,
    get_df_targets,
    get_project_eui_targets,
    get_all_buildings_yearly_eui_uknzcb_targets,
    get_eui_targets,
    get_eui_target,
    calc_area_weighted_target
)


project_revision_id, object_id = 109, OBJECT_ID_BUILDING_AREA  # crystal palace - app.project_revision_id


# Show you working...

## Calc Inputs

In [7]:
df_eui = nzdata.energy_use_intensity
df_project_buildings = get_project_building_data(project_revision_id)
df_area = get_area_data(project_revision_id, object_id)
df_project_energy_targets = get_project_energy_targets(project_revision_id=project_revision_id)

df_eui.to_csv("df_eui.csv")
df_project_buildings.to_csv("df_project_buildings.csv")
df_area.to_csv("df_area.csv")
df_project_energy_targets.to_csv("df_project_energy_targets.csv")

import ipywidgets as w
from ipydatagrid import DataGrid

w.Tab([
    DataGrid(df_project_buildings, auto_fit_columns=True), DataGrid(df_area, auto_fit_columns=True), DataGrid(df_project_energy_targets, auto_fit_columns=True), DataGrid(df_eui, auto_fit_columns=True),
],
    titles=["df_project_buildings", "df_area", "df_project_energy_targets", "df_eui"]
)

## Project Applicable EUI Targets (not area weighted)

In [4]:
df_area = attach_target_year_to_area(df_area, df_project_buildings)
eui_uknzcb_project = get_project_eui_targets(df_area, df_eui)
eui_uknzcb_project

Year                            BuildingType ConstructionMethod  \
0  2026                   Sport & Leisure - Dry       retrofit-in-one-go   
1  2026                   Sport & Leisure - Wet       retrofit-in-one-go   
2  2026               Sport & Leisure - Fitness       retrofit-in-one-go   
3  2026  Storage & Distribution - Unconditioned       retrofit-in-one-go   
4  2026  Storage & Distribution - Unconditioned       retrofit-in-one-go   

   Target  TargetYear   BuildingName  
0   206.0        2026  Main Building  
1   490.0        2026  Main Building  
2   275.0        2026  Main Building  
3    34.0        2026  Main Building  
4    34.0        2026  Main Building

In [49]:
import duckdb
area, eui = df_area, eui_uknzcb_project

duckdb.register("area", area)
duckdb.register("eui", eui)
q = """
SELECT      area.BuildingType, GrossInternalArea, Target
FROM        area
INNER JOIN  eui
ON          area.ConstructionMethod=eui.ConstructionMethod
AND         area.BuildingType=eui.BuildingType"""


df = duckdb.sql(q).df()
df

BuildingType  GrossInternalArea  Target
0                   Sport & Leisure - Dry             4595.0   206.0
1  Storage & Distribution - Unconditioned              324.0    34.0
2               Sport & Leisure - Fitness             1809.0   275.0
3                   Sport & Leisure - Wet             3764.0   490.0
4  Storage & Distribution - Unconditioned              911.0    34.0
5  Storage & Distribution - Unconditioned              324.0    34.0
6  Storage & Distribution - Unconditioned              911.0    34.0

In [ ]:
p_trgt, p_area = df.Target, df.GrossInternalArea
calc_area_weighted_target(p_trgt, p_area)

In [6]:
df_nzc_standard_building_year_targets = get_all_buildings_yearly_eui_uknzcb_targets(
    df_area,
    df_eui,
)
df_nzc_standard_building_year_targets

Year      Target   BuildingName       TargetName
0   2025  297.877313  Main Building  UK NZC Standard
1   2026  292.063054  Main Building  UK NZC Standard
2   2027  286.248794  Main Building  UK NZC Standard
3   2028  280.275892  Main Building  UK NZC Standard
4   2029  274.461633  Main Building  UK NZC Standard
5   2030  268.647373  Main Building  UK NZC Standard
6   2031  262.674472  Main Building  UK NZC Standard
7   2032  256.860212  Main Building  UK NZC Standard
8   2033  251.045953  Main Building  UK NZC Standard
9   2034  245.073051  Main Building  UK NZC Standard
10  2035  239.258792  Main Building  UK NZC Standard
11  2036  233.444532  Main Building  UK NZC Standard
12  2037  227.471630  Main Building  UK NZC Standard
13  2038  221.657371  Main Building  UK NZC Standard
14  2039  215.843111  Main Building  UK NZC Standard
15  2040  209.870210  Main Building  UK NZC Standard
16  2050  209.870210  Main Building  UK NZC Standard

NameError: name 'df' is not defined

In [43]:
get_eui_target(df_area, eui_uknzcb_project)

np.float64(266.8448330432031)

In [ ]:
    duckdb.register("area", area)
    duckdb.register("eui", eui)
    q = """
    SELECT      GrossInternalArea, Target
    FROM        area
    INNER JOIN  eui
    ON          area.ConstructionMethod=eui.ConstructionMethod
    AND         area.BuildingType=eui.BuildingType"""

    if filter_year:
        and_yr = """
        AND         area.TargetYear=eui.Year"""
        q = q + and_yr
    df = duckdb.sql(q).df()
    p_trgt, p_area = df.Target, df.GrossInternalArea

In [34]:
df_area

BuildingId  TypeSpecId  InstanceReference  GrossInternalArea  \
0     BLDG-1        2919                  1             4595.0   
1     BLDG-1        2919                  2             1809.0   
2     BLDG-1        2919                  3             3764.0   
3     BLDG-1        2919                  4              324.0   
4     BLDG-1        2919                  5              911.0   

                             BuildingType  IsNewBuild   Id  \
0                   Sport & Leisure - Dry       False  422   
1               Sport & Leisure - Fitness       False  423   
2                   Sport & Leisure - Wet       False  424   
3  Storage & Distribution - Unconditioned       False  425   
4  Storage & Distribution - Unconditioned       False  426   

  ConstructionMethod   BuildingName  
0       retrofit-in-one-go  Main Building  
1       retrofit-in-one-go  Main Building  
2       retrofit-in-one-go  Main Building  
3       retrofit-in-one-go  Main Building  
4       retrofit-in-one-go  Main Building

NameError: name 'get_eui_targets' is not defined

In [19]:
get_project_targets(df_area, df_eui, df_project_energy_targets)

BinderException: Binder Error: Values list "area" does not have a column named "TargetYear"

In [53]:

if not df_area.empty:
    (
        df_area_summary,
        di_area_weights,
        di_project_area_weights,
        df_nzc_standard_building_year_targets,
        df_all_building_targets,
        df_all_project_targets,
    ) = get_area_and_target_summaries(
        df_project_buildings, df_area, df_project_energy_targets, df_eui
    )

BinderException: Binder Error: Ambiguous reference to column name "TargetYear" (use: "_area.TargetYear" or "_bldg.TargetYear")

Year                            BuildingType ConstructionMethod  \
0  2026                   Sport & Leisure - Dry       retrofit-in-one-go   
1  2026                   Sport & Leisure - Wet       retrofit-in-one-go   
2  2026               Sport & Leisure - Fitness       retrofit-in-one-go   
3  2026  Storage & Distribution - Unconditioned       retrofit-in-one-go   
4  2026  Storage & Distribution - Unconditioned       retrofit-in-one-go   

   Target  TargetYear   BuildingName  
0   206.0        2026  Main Building  
1   490.0        2026  Main Building  
2   275.0        2026  Main Building  
3    34.0        2026  Main Building  
4    34.0        2026  Main Building

In [ ]:
get_project_eui_targets

In [7]:
di_targets = get_project_targets(df_area, df_eui, df_project_energy_targets)
di_targets

BinderException: Binder Error: Values list "area" does not have a column named "TargetYear"

In [52]:

df_eui = get_khwr_per_m2_gia_only(df_eui)  # TODO: remove non GIA from data
df_area = attach_target_year_to_area(df_area, df_project_buildings)
df_area_summary = get_area_summary(df_area)
di_area_weights = get_area_weights(df_area_summary)
di_project_area_weights = get_project_area_weights(df_area_summary)

df_nzc_standard_building_year_targets = get_all_buildings_yearly_eui_uknzcb_targets(
    df_area,
    df_eui,
)

di_targets = get_project_targets(df_area, df_eui, df_project_energy_targets)
df_targets = get_df_targets(di_targets)
df_all_project_targets = df_targets[["Project"]].reset_index()
df_all_project_targets.columns = ["TargetName", "ProjectTarget"]

df_all_building_targets = df_targets.copy(deep=True)
del df_all_building_targets["Project"]
df_all_building_targets = pd.melt(
    df_all_building_targets.reset_index(),
    id_vars=["TargetName"],
    var_name="BuildingName",
    value_name="Target",
)

BinderException: Binder Error: Ambiguous reference to column name "TargetYear" (use: "_area.TargetYear" or "_bldg.TargetYear")

In [23]:
get_project_eui_targets(df_area, df_eui)

Year                            BuildingType ConstructionMethod  \
0  2026                   Sport & Leisure - Dry       retrofit-in-one-go   
1  2026                   Sport & Leisure - Wet       retrofit-in-one-go   
2  2026               Sport & Leisure - Fitness       retrofit-in-one-go   
3  2026  Storage & Distribution - Unconditioned       retrofit-in-one-go   
4  2026  Storage & Distribution - Unconditioned       retrofit-in-one-go   

   Target  TargetYear   BuildingName  
0   206.0        2026  Main Building  
1   490.0        2026  Main Building  
2   275.0        2026  Main Building  
3    34.0        2026  Main Building  
4    34.0        2026  Main Building

In [ ]:
get_eui_targets(df_area, eui_uknzcb_project)

In [13]:
get_project_eui_targets(
        df_area,
        df_eui,
        filter_year=True,
    )  # filter out targets from ref data relevant to project

NameError: name 'get_project_eui_targets' is not defined

In [12]:
df_project_energy_targets

Empty DataFrame
Columns: []
Index: []

In [11]:
di_targets

{'UK NZC Standard': {'Main Building': np.float64(266.8448330432031),
  'Project': np.float64(266.8448330432031)}}